# Importation 

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Merge all
df_merged = pd.read_csv('Merged_players_Whoscore.csv')

In [ ]:
#Load forecasted rating dataset
forecastedPlayerDataframe = pd.read_csv("Forecasted_Ratings_2023-2025.csv") 

In [ ]:
def get_player_df(name):
    player_df  = combinedPlayerDataframe.loc[(combinedPlayerDataframe.Name == name) & (combinedPlayerDataframe.Season == 2018)]
    return player_df

In [ ]:
def get_current_squad_df(team):
    squad_df = combined_player_data_frame.loc[(combined_player_data_frame.Team == team) & (combined_player_data_frame.Season == 2018)]
    return squad_df

In [ ]:
#Creating a seperate DF for players who are between the age of 23-28 in 2018

youngPlayers2018Df = forecastedPlayerDataframe.loc[(forecastedPlayerDataframe.Season == 2022) 
                                                   & ((forecastedPlayerDataframe.Age > 22) 
                                                       & (forecastedPlayerDataframe.Age < 29))]

In [ ]:
#Storing the names of the young players on to list

youngPlayerNames = youngPlayers2022Df.Name.tolist()

In [ ]:
#Creating a seperate dictionary entry for every player
#Creating a new row of data for every player with the mean of all their stats
#Rating has been replaced by the forecasted rating average

young_players_ = {}

for name in youngPlayerNames:
    playerMeanRating = forecastedPlayerDataframe.loc[(forecastedPlayerDataframe.Name == name)]['Rating'].mean()
    playerDf = combinedPlayerDataframe.loc[(combinedPlayerDataframe.Name == name)]
    playerDf = playerDf.sort_values(by='Age', ascending=True)
    playerDf.loc['average'] =  playerDf.mean()
    playerDf.loc['average', 'Name'] = name
    playerDf.loc['average', 'Team'] = playerDf['Team'][playerDf.index[-2]]
    playerDf.loc['average', 'Position'] = playerDf['Position'][playerDf.index[-2]]
    playerDf.loc['average', 'League'] = playerDf['League'][playerDf.index[-2]]
    playerDf.loc['average', 'Rating'] = playerMeanRating
    playerDf = playerDf.iloc[-1:]
    young_players_[name] = playerDf

In [ ]:
young_players_['Leroy Sané']

# Predict remplacement for older players

In [ ]:
#Choose club

club_name = 'Real Madrid'

In [ ]:
#Creating a sperate DF for older players

olderPlayersDf = combinedPlayerDataframe.loc[(combinedPlayerDataframe.Team == club_name) 
                                            & (combinedPlayerDataframe.Season == 2022) 
                                            & (combinedPlayerDataframe.Age > 30)]

In [ ]:
#Creating a seperate dictionary to store the average stat data of the best years (24-29) of the older players

older_players_ = {}

for index, row in olderPlayersDf.iterrows():
    
    currentDf = combinedPlayerDataframe.loc[(combinedPlayerDataframe.Name == row['Name']) 
                                            & (combinedPlayerDataframe.Age > 23)  
                                            & (combinedPlayerDataframe.Age < 29)]
    currentDf = currentDf.sort_values(by='Age', ascending=True)
    currentDf.loc['average'] =  currentDf.mean()
    currentDf.loc['average', 'Name'] = row['Name']
    currentDf.loc['average', 'Team'] = currentDf['Team'][currentDf.index[-2]]
    currentDf.loc['average', 'Position'] = currentDf['Position'][currentDf.index[-2]]
    currentDf.loc['average', 'League'] = currentDf['League'][currentDf.index[-2]]
    currentDf = currentDf.iloc[-1:]
    older_players_[row['Name']] = currentDf

In [ ]:
#Recommending young replacements within the age of (23-28)

li = []

old_player_ = {}

for name in older_players_:
    position = older_players_[name].Position.tolist()[0]
    li.append(older_players_[name])
    youngDf = postionYoungPlayers_[position]
    youngDf = youngDf[youngDf.Team != club_name]
    li.append(youngDf)
    
    combinedDf = pd.concat(li)
    
    stats = combinedDf.copy(deep=True)
    stats.drop(['Name', 'Team', 'League', 'Position'], axis=1, inplace=True)
    
    stats_with_name = stats
    stats = stats.dropna()
    stats_with_name = stats_with_name.dropna()
    stats_with_name["Name"] = combinedDf["Name"]
    
    stats_with_name = stats_with_name.reset_index(drop=True)
    
    from sklearn.preprocessing import StandardScaler
    from sklearn.neighbors import NearestNeighbors
    from sklearn.exceptions import DataConversionWarning

    import warnings
    warnings.filterwarnings(action="ignore",category=DataConversionWarning)
    
    scaler = StandardScaler()
    
    X = scaler.fit_transform(stats)
    
    #Limiting the number of similar profiles(clusters) to be found to 3. (4 in total with the current player being one)
    knn = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X)

    player_indexes = knn.kneighbors(X)[1]
    
    print("")
    print("Older Player: " + name)
    
    old_player_[name] = get_player_df(name)
    
    index = stats_with_name[stats_with_name["Name"]==name].index.tolist()[0]
    count = 1
    for i in player_indexes[index][1:]:
            print('Option ' + str(count) + ': ' + stats_with_name.iloc[i]["Name"])
            
            old_player_[name] = old_player_[name].append(get_player_df(stats_with_name.iloc[i]["Name"]))
            
            count += 1

    
    li=[]
# print(old_player_['Luka Modric'])

In [ ]:
def age_modification(df_list):
    for df in df_list:
        for index, row in df.iterrows():
            birth_year = 2023 - int(row["Age"])
            age = int(row["Saison"]) - birth_year
            df.at[index, 'Age'] = age
    return df_list

In [ ]:
age_modification(df_list)